In [1]:
import datetime
import requests
import credentials
from web3 import Web3
from moralis import evm_api

In [2]:
ETHERSCAN_KEY = credentials.etherscan_key
QUICKNODE_KEY = credentials.quicknode_key
MORALIS_KEY = credentials.moralis_key

### Get Contract Creation Txn Hash

In [4]:
def get_contract_creation_txh(contract_address: []):
    url = 'https://api.etherscan.io/api'
    params = {
        'module': 'contract',
        'action': 'getcontractcreation',
        'contractaddresses': contract_address,
        'apikey': ETHERSCAN_KEY
    }
    response = requests.get(url, params=params)
    data = response.json()
    if data['status'] != '1':
        print("Error: ", data['message'])
        return None
    else:
        return data['result']

def get_txn_by_hash(txn_hash):
    url = 'https://api.etherscan.io/api'
    params = {
        'module': 'proxy',
        'action': 'eth_getTransactionByHash',
        'txhash': txn_hash,
        'apikey': ETHERSCAN_KEY
    }
    response = requests.get(url, params=params)
    return response.json()

def get_recent_block():
    url = 'https://api.etherscan.io/api'
    params = {
        'module': 'proxy',
        'action': 'eth_blockNumber',
        'apikey': ETHERSCAN_KEY
    }
    response = requests.get(url, params=params)
    return response.json()

def get_block_by_num(number):
    url = 'https://api.etherscan.io/api'
    params = {
        'module': 'proxy',
        'action': 'eth_getBlockByNumber',
        'tag': number,
        'boolean': 'true',
        'apikey': ETHERSCAN_KEY
    }
    response = requests.get(url, params=params)
    return response.json()

def get_token_block_num(contract_address):
    txn_hash = get_contract_creation_txh(contract_address)
    txn = get_txn_by_hash(txn_hash[0]['txHash']) #array
    block_start = get_block_by_num(txn['result']['blockNumber'])
    block_time_start = int(block['result']['timestamp'][2:], 16)

    block_now = get_recent_block()
    block_now = int(block_now['result'][2:], 16)

    data = {
        'block_now_num': block_now,
        'block_start_num': int(block_start['result']['number'][2:], 16),
        'block_time_start': block_time_start
    }
    return data

In [ ]:
contract_address = Web3.to_checksum_address("0x586a7cFE21E55ec0E24f0bfB118F77fE4cA87bab")
WETH = Web3.to_checksum_address("0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2")
token_data = get_token_block_num(contract_address)
block_start_num = token_data['block_start_num']
block_now_num = token_data['block_now_num']
block_time_start = token_data['block_time_start']

now_time = int(datetime.datetime.now().timestamp())

time_difference = datetime.datetime.fromtimestamp(now_time) - datetime.datetime.fromtimestamp(block_time_start)
days = time_difference.days
hours, remainder = divmod(time_difference.seconds, 3600)
minutes, seconds = divmod(remainder, 60)

print(f"Token existed for {days} days, {hours} hours, {minutes} minutes, {seconds} seconds")

historicalPrice = []
for to_block in range(block_start_num, block_now_num, 1000):
  params = {
    "address": contract_address,
    "chain": "eth",
    "to_block": to_block
  }
  result = evm_api.token.get_token_price(
    api_key=MORALIS_KEY,
    params=params,
  )

  historicalPrice.append(result)


In [ ]:
historicalPrice